# 레시피 추천 시스템

https://chan-chance.tistory.com/22

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from gensim.models import Word2Vec

In [8]:
coupang = pd.read_csv('../../data/coupang_filtering.csv')
coupang.drop('Unnamed: 0', axis=1, inplace=True)

# ckg = pd.read_csv('../../data/ckg_drop.csv')
# ckg.drop('Unnamed: 0', axis=1, inplace=True)

# #결측치 제거
# ckg.dropna(inplace=True)
# ckg.reset_index(inplace=True, drop=True)

## 만개의 레시피에서 조회수 기반 스크랩수 가중치 계산

In [3]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating

    ( (각 조회수 / (각 조회수+최소조회수)) * 각 스크랩수 ) + ( (최소조회수/(각 조회수+최소조회수)) * 평균스크랩수 )
    
    Returns: pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [ ]:
# # calcuate input parameters
# C = np.mean(ckg['스크랩수'])
# m = np.percentile(ckg['조회수'], 70)
# vote_count = ckg[ckg['조회수'] >= m]
# R = ckg['스크랩수']
# v = ckg['조회수']

# ckg['weighted_clipping'] = weighted_rating(v,m,R,C)
# ckg.to_csv('../../data/ckg_weighted.csv') #가중치 추가 저장
# ckg.head()

In [10]:
#가중치까지 계산한 df을 불러옴
ckg = pd.read_csv('../../data/ckg_weighted.csv')
ckg.drop('Unnamed: 0', axis=1, inplace=True)
#word2vec 사용을 위한 데이터 포멧 변경
ckg['요리재료_전처리'] = ckg['요리재료_전처리'].apply(lambda x : x.split(', ')) # 1차원 list 형태
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
0,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]",320.630665
1,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루]",289.708247
2,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,"[파스타면, 토마토, 토마토, 양파, 다진마늘, 피망, 올리브유]",216.670718
3,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일, 녹말가루, 물, 계란 노...",282.469535
4,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,"[닭, 통마늘, 다진마늘, 화이트와인또는우유, 올리브유또는버터, 마늘솔트또는소금, ...",255.980639


# word2vec 기반
- tfidf와 비교했을 때 w2v의 코사인이 카테고리 분류하지 않아도 유사한 레시피를 잘 찾아줌
- w2v에서 CBOW와 Skip-Gram 사이의 정확도는 정확한 데이터가 있어야 비교 가능할 듯

In [6]:
##### input 값 전처리 후 list에 단어별 저장 #####
def get_product_list(product_df):
    # 상품명 전처리
    product = product_df.상품.values.tolist()
    product_list = ' '.join(s for s in product)
    product_list = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』]', '', product_list).split(' ') #특수문자제거(만개의레시피 데이터엔 특수문자가 없어서 인식 불가)
    product_list = list(filter(None, product_list)) #null 제거(안 할 경우 레시피 전체가 선택되어 작동 불가)
    
    #구성정보 전처리
    str_list = product_df.구성정보_전처리.values.tolist()
    ingredients_list = ' '.join(s for s in str_list).split(',')
    
    return product_list, ingredients_list


##### input 상품과 동일한 요리명의 레시피만 추출 #####
def get_recipe_df(product_list):
    recipe_df = pd.DataFrame()
    for word in product_list:
        #recipe = ckg[ckg['요리명']==word]
        recipe = ckg[ckg['요리명'].str.contains(word)]
        recipe_df = pd.concat([recipe_df, recipe])
    return recipe_df


###### 단어 벡터 평균구하기 ######
def vectors(embedding, embedding_model):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(embedding_model.wv[word]) 
            count+=1
        except : continue

    word_embedding2=np.array(word_embedding)
    v=word_embedding2.sum(axis=0)  
    
    if type(v)!=np.ndarray:
        return np.zeros(100)
    
    return(v/count)


###### 최종 알고리즘 ######
def wv_mealkit(sim,mealkit_index,top_n,recipe_df):
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]

    # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
    similar_indexes=sim_sorted_ind[mealkit_index,1:(top_n+1)]
    recipe = recipe_df.iloc[similar_indexes].sort_values(by='weighted_clipping',ascending=False)
    
    return recipe


###### 레시피 only 재료 추출 ######
def get_recipe_only(ingredients_list, sim_mealkit_df):
    ingredients_set = set(ingredients_list) #요리재료의 중복값 제거
    print('ingredients = ', ingredients_set)
    
    recipe_ingredient = []
    for items in sim_mealkit_df.요리재료_전처리:
        for item in items:
            # input 구성정보와 요리재료가 일치하면 제외
            if item in ingredients_set: 
                items.remove(item)
        #레시피에만 있는 요리재료 추가
        recipe_ingredient.append(items)
    return recipe_ingredient

## 실행코드

In [29]:
# coupang에서 input 값의 df 추출
num = 299
product_df = coupang.iloc[num:num+1, :]
display(product_df)

# input 상품명 및 구성정보 전처리 후 list로 추출
product_list, ingredients_list = get_product_list(product_df)

try:
    # input 상품명과 일치하는 레시피만 추출
    recipe_df = get_recipe_df(product_list)

    # 일치하는 레시피의 요리재료 전체 append
    word = [words for words in recipe_df.요리재료_전처리]

    # word list의 가장 마지막에 input 구성정보 append
    word.append(ingredients_list)

    # word2vec 모델 생성
    wv_model = Word2Vec(sentences=word, vector_size=100, window=5, min_count=1,  sg=1, batch_words=1000)

    #레시피 별 벡터 평균 추출
    wordvec=[vectors(x, wv_model) for x in word]

    # 코사인유사도 계산
    w2v_sim=cosine_similarity(wordvec,wordvec)

    # 코사인유사도 행렬에서 input 상품의 인덱스 저장
    mealkit_index = len(w2v_sim)-1

    #input 상품과 유사한 레시피 df 추출
    sim_mealkit_df = wv_mealkit(w2v_sim, mealkit_index, 5, recipe_df)
    display(sim_mealkit_df) 
    
except: 
    print("유사한 레시피가 업습니다.")


#레시피에만 있는 재료 추출
recipe_ingredients = get_recipe_only(ingredients_list, sim_mealkit_df)
recipe_ingredients = list(filter(None, recipe_ingredients)) # input 구성정보와 요리재료가 모두 일치하는 레시피 제외
recipe_ingredients

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
299,7,6431985897,13884840614,81134595803,복선당 안동식 야채듬뿍 찜닭 & 떡 (냉동),"복선당 안동식 야채듬뿍 찜닭 & 떡 (냉동), 1260g, 1개",14900,0,14900,"(100g당 1,183원)",0.0,0,1,"수비드 찜닭, 소스, 떡, 감자 가공품, 레드캐롯 가공품, 마늘로 구성되어 있습니다.","수비드 찜닭,소스,떡,마늘",닭


C:\Users\user\AppData\Local\Temp\ipykernel_14204\3811083497.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  recipe = ckg[ckg['요리명'].str.contains(word)]


,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
35209,6864113,꼬들꼬들 무채 식감이 좋아요~ 무말랭이떡볶이,무말랭이떡볶이,1369,1,26,볶음,일상,채소류,밥/죽/떡,무말랭이를 넣고 떡볶이를 했더니 맛도 맛이지만 꼬들꼬들한 식감이 있어 먹는 재미가 ...,[재료] 고운고추가루 3큰술| 설탕 3큰술| 다시마 멸치육수 1/4컵 |무말랭이 1...,"[고운고추가루, 설탕, 다시마 멸치육수, 무말랭이, 절편, 사각어묵, 냉동만두, 다...",354.681264
66533,6912850,숟가락으로 국물과 함께 떠 먹는 오징어 국물떡볶이,오징어국물떡볶이,1685,0,30,끓이기,일상,해물류,밥/죽/떡,오랜 지인분께서 판매용이 아닌 식구들 나눠 드신다고 생물 오징어를 사다가 손질해 직...,[재료] 반건조오징어 1마리| 가래떡 600g| 다시마 멸치육수 4컵| 소금 1/3...,"[반건조오징어, 가래떡, 다시마 멸치육수, 소금, 고운고추가루, 설탕, 다시마 멸치...",348.258191
15941,6797534,양념게장 기름떡볶이,양념게장기름떡볶이,1933,2,18,볶음,간식,쌀,밥/죽/떡,양념게장 양념은 무진장 맛있고 칼칼한데 이 양념을 어떻게 활용할 것인가~ 생각해보...,[재료] 떡| 카놀라유 [양념] 양념게장양념,"[떡, 카놀라유, 양념게장양념]",341.613385
41820,6874992,불닭볶음떡볶이,불닭볶음떡볶이,2554,0,48,볶음,초스피드,쌀,밥/죽/떡,라면스프를 활용해서 맛있는 떡볶이를 만들 수 있습니다.빠른 시간안에 유명 떡볶이집보...,[재료] 떡 150그램| 오뎅 100그램| 버섯 약간| 계란 1개| 파 약간| 피자...,"[오뎅, 버섯, 계란, 파, 피자치즈, 삼양불닭볶음탕면 스프, 설탕]",333.837030
30275,6853996,국민간식 초간단 떡볶이/크림 떡볶이,크림떡볶이,26220,31,270,볶음,초스피드,쌀,밥/죽/떡,아이 어른 남녀노소 모두 좋아하는 국민간식 떡볶이를 크림소스 넣고 하얗게 만들어 ...,[재료] 떡볶이떡 400g| cj 크림 파스타소스 200g| 올리브유 1스푼| 양파...,"[떡볶이떡, 크림, 올리브유, 양파, 대파, 소금, 물, 허브솔트]",310.037879


ingredients =  {'수비드 찜닭', '마늘', '소스', '떡'}


[['고운고추가루',
  '설탕',
  '다시마 멸치육수',
  '무말랭이',
  '절편',
  '사각어묵',
  '냉동만두',
  '다시마 멸치육수',
  '소금',
  '양파',
  '대파'],
 ['반건조오징어',
  '가래떡',
  '다시마 멸치육수',
  '소금',
  '고운고추가루',
  '설탕',
  '다시마 멸치육수',
  '구운 계란',
  '양파',
  '대파'],
 ['카놀라유', '양념게장양념'],
 ['오뎅', '버섯', '계란', '파', '피자치즈', '삼양불닭볶음탕면 스프', '설탕'],
 ['떡볶이떡', '크림', '올리브유', '양파', '대파', '소금', '물', '허브솔트']]

> 이슈
1. 320번, 75번 일치하는 레시피가 없을 경우 전혀 다른 레시피 또는 아무것도 안 나옴
2. '계란  개 소금' 등 추가 전처리 필요
3. 양념 재료는 제외할 수 있는 재료 사전 필요
4. 쿠팡 재료의 재료명 일치 필요